In [ ]:
import requests, json, io, urllib
from zipfile import ZipFile
#import zipfile
#from contextlib import closing
import urllib.request as ur
import pandas as pd
import subprocess
from simpledbf import Dbf5
from bis2 import gc2

q='''https://www.sciencebase.gov/catalog/items?filter=tags={%22scheme%22:%22BIS%22,%22name%22:%22NHDPlusV2.1%22}&fields=files,id,tags&format=json'''
#Returns 
nhdItems = requests.get(q).json()

# Set up the actions/targets for this particular instance
thisRun = {}
thisRun["instance"] = "DataDistillery"
thisRun["db"] = "BCB"
thisRun["baseURL"] = gc2.sqlAPI(thisRun["instance"],thisRun["db"])
thisRun["schema"] = "sfr"
thisRun["commitToDB"] = False


#Query comids already in the table
q_recordToSearch = "SELECT comid as lookup FROM nhd.nhdplusv2_plusflowlinevaa " 
recordToSearch = requests.get(thisRun["baseURL"]+"&q="+q_recordToSearch).json()
p = recordToSearch['features']
lookup = []
#for record returned, add record to list
for f in p:
    string = f['properties']['lookup']
    if string.startswith('"') and string.endswith('"'):
        string = string[1:-1]  #For some reason these values are coming back with double quotes, this removes outer quote
        lookup.append(string)
lenList = len(lookup)
cntExisting = 0 
existingList = []
    


for item in nhdItems['items']:
    #Identify which NHD Region via tags 
    for tag in item['tags']:
        if 'Reg' in tag['name']:
            region = tag['name']
    #Look at files and find NHDPlusAttributes File
    for file in item['files']:
        fileName = file['name']
        if 'NHDPlusAttributes' in fileName:
            #Download file
            print ('Retrieving region ' + region + ', file:' + fileName)
            fileUrl = file['url']
            ur.urlretrieve(fileUrl, fileName)
            #Unzip file.  Ideally this will be transformed to work in memory
            subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + fileName)
            
            #Convert dbf to dataframe
            dbf = Dbf5('PlusFlowLineVAA.dbf')
            df = dbf.to_dataframe()
            for row in df.itertuples():
                if row.ComID in lookup:
                    continue
                else:
                    q = "insert into nhd.nhdplusv2_plusflowlinevaa \
                    (comid, fdate, streamleve, streamorde, streamcalc, fromnode, tonode, hydroseq, levelpath1, \
                    pathlength, terminalpat, arbolatesu, divergence, startflag, terminalfl, dnlevel,  thinnercod, \
                    uplevelpat, uphydroseq, dnlevelpat, dnminorhyd, dndraincou, dnhydroseq, frommeas, tomeas, \
                    reachcode, lengthkm, fcode, rtndiv, outdiv, diveeffect, vpuin, vpuout, areasqkm, totdasqkm, \
                    divdasqkm, tidal, totma, wbareatype) VALUES \
                    ('" + str(row.ComID) + "' ,'" + str(row.Fdate) + "' ,'" + str(row.StreamLeve) + "' ,'"  \
                    + str(row.StreamOrde) + "' ,'" + str(row.StreamCalc) + "' ,'" + str(row.FromNode) + "' ,'" \
                    + str(row.ToNode) + "' ,'" + str(row.Hydroseq) + "' ,'" + str(row.LevelPathI) +  "' ,'" \
                    + str(row.Pathlength)+ "' ,'" + str(row.TerminalPa) + "' ,'" + str(row.ArbolateSu) + "' ,'" \
                    + str(row.Divergence) + "' ,'" + str(row.StartFlag) + "' ,'" + str(row.TerminalFl) + "' ,'" \
                    + str(row.DnLevel) + "' ,'" +   str(row.ThinnerCod) + "' ,'" +   str(row.UpLevelPat) + "' ,'" \
                    + str(row.UpHydroseq) + "' ,'" + str(row.DnLevelPat) + "' ,'" + str(row.DnMinorHyd) + "' ,'" \
                    + str(row.DnDrainCou) + "' ,'" + str(row.DnHydroseq) + "' ,'" + str(row.FromMeas) + "' ,'" \
                    + str(row.ToMeas) + "' ,'" + str(row.ReachCode) + "' ,'" + str(row.LengthKM) + "' ,'" \
                    + str(row.Fcode) + "' ,'" + str(row.RtnDiv) + "' ,'" + str(row.OutDiv) + "' ,'" \
                    + str(row.DivEffect) + "' ,'" + str(row.VPUIn) + "' ,'" + str(row.VPUOut) + "' ,'" \
                    + str(row.AreaSqKM) + "' ,'" + str(row.TotDASqKM) + "' ,'" + str(row.DivDASqKM) + "' ,'" \
                    + str(row.Tidal) + "' ,'" + str(row.TOTMA) + "' ,'" + str(row.WBAreaType) + "')"
                    gc2Key = gc2.gc2Keys["datadistillery_bcb"]
                    payload = "q=%s&key=%s"%(q,gc2Key)
                    url= gc2.baseURLs["sqlapi_datadistillery_bcb"]
                    finalUrl = url + '?q=' +q + '&key=' + gc2Key
                                
                    try:
                        #r = requests.post(url,data=payload)
                        r = requests.get(finalUrl)
                        #r.get(finalUrl).json()
                        status = str(r.status_code)
                        if r.status_code == 200:
                            continue
                        else:
                            print (str(row.ComID) + ' failed with code: ' + status)
                    except requests.exceptions.HTTPError:
                        print (str(row.ComID) + ' failed with code: ' + status)
   

Retrieving region Reg9, file:NHDPlusAttributes.zip
7030747 failed with code: 503
7030199 failed with code: 503
7030195 failed with code: 503
7030131 failed with code: 503
7030115 failed with code: 503
7030235 failed with code: 503
7030751 failed with code: 503
7030233 failed with code: 503
7030827 failed with code: 503
7030213 failed with code: 503
7030823 failed with code: 503
7030157 failed with code: 503
7030171 failed with code: 503
7030175 failed with code: 503
7030183 failed with code: 503
7030819 failed with code: 503
7030621 failed with code: 503
7030039 failed with code: 503
7029943 failed with code: 503
7029951 failed with code: 503
7030005 failed with code: 503
7029985 failed with code: 503
7030807 failed with code: 503
7029941 failed with code: 503
7029879 failed with code: 503
7030803 failed with code: 503
7029895 failed with code: 503
7030573 failed with code: 503
7030559 failed with code: 503
7030541 failed with code: 503
7029795 failed with code: 503
7030561 failed with

7027113 failed with code: 503
7027139 failed with code: 503
7027179 failed with code: 503
7027115 failed with code: 503
7027097 failed with code: 503
7027043 failed with code: 503
7027175 failed with code: 503
7027141 failed with code: 503
7027159 failed with code: 503
7027219 failed with code: 503
7027231 failed with code: 503
909020801 failed with code: 503
7027233 failed with code: 503
7027225 failed with code: 503
7027183 failed with code: 503
7027181 failed with code: 503
7027223 failed with code: 503
7027209 failed with code: 503
7027167 failed with code: 503
7030389 failed with code: 503
7030385 failed with code: 503
7030383 failed with code: 503
7030677 failed with code: 503
7030673 failed with code: 503
7030853 failed with code: 503
7030669 failed with code: 503
7030361 failed with code: 503
7030851 failed with code: 503
7030663 failed with code: 503
7030661 failed with code: 503
7030359 failed with code: 503
7030365 failed with code: 503
7027093 failed with code: 503
7027155 

14145047 failed with code: 503
909020858 failed with code: 503
909020859 failed with code: 503
14144975 failed with code: 503
14144973 failed with code: 503
14144951 failed with code: 503
14144919 failed with code: 503
14144095 failed with code: 503
14143897 failed with code: 503
14143871 failed with code: 503
14143855 failed with code: 503
14143747 failed with code: 503
14143849 failed with code: 503
14143743 failed with code: 503
14143717 failed with code: 503
14143599 failed with code: 503
14144035 failed with code: 503
14145171 failed with code: 503
14144911 failed with code: 503
14144903 failed with code: 503
14144901 failed with code: 503
14144897 failed with code: 503
14143921 failed with code: 503
14144867 failed with code: 503
14143923 failed with code: 503
14143893 failed with code: 503
14144081 failed with code: 503
14144071 failed with code: 503
14143699 failed with code: 503
14143675 failed with code: 503
14143623 failed with code: 503
14143595 failed with code: 503
141435

14143443 failed with code: 503
14143383 failed with code: 503
14143389 failed with code: 503
14143393 failed with code: 503
14143401 failed with code: 503
14143397 failed with code: 503
14143403 failed with code: 503
14143411 failed with code: 503
14143425 failed with code: 503
14143439 failed with code: 503
14143453 failed with code: 503
14143463 failed with code: 503
14143461 failed with code: 503
14143455 failed with code: 503
14143477 failed with code: 503
14143473 failed with code: 503
14143483 failed with code: 503
14143501 failed with code: 503
14143493 failed with code: 503
14143503 failed with code: 503
14143499 failed with code: 503
14143997 failed with code: 503
14143487 failed with code: 503
14143491 failed with code: 503
14143497 failed with code: 503
14143523 failed with code: 503
14143527 failed with code: 503
14143475 failed with code: 503
14143467 failed with code: 503
14143511 failed with code: 503
14143517 failed with code: 503
14144001 failed with code: 503
14143537

In [ ]:
print ()

In [9]:
df.head()

,ComID,Fdate,StreamLeve,StreamOrde,StreamCalc,FromNode,ToNode,Hydroseq,LevelPathI,Pathlength,...,OutDiv,DivEffect,VPUIn,VPUOut,AreaSqKM,TotDASqKM,DivDASqKM,Tidal,TOTMA,WBAreaType
0,14045663,2012-01-06,6,1,1,840025648,840013505,840024257,840008673,577.289,...,0,0,0,0,8.4213,8.4213,8.4213,0,0.180459,NaN
1,14045631,2012-01-06,7,1,1,840025642,840013502,840024263,840024263,576.085,...,0,0,0,0,1.7793,1.7793,1.7793,0,0.194315,NaN
2,14045635,2012-01-06,7,1,1,840025643,840013505,840024262,840024262,577.289,...,0,0,0,0,1.4823,1.4823,1.4823,0,0.136649,NaN
3,14045577,2012-01-06,7,1,1,840025632,840013493,840024273,840024273,573.474,...,0,0,0,0,2.3364,2.3364,2.3364,0,0.186858,NaN
4,14045581,2012-01-06,7,1,1,840013503,840013491,840015494,840015494,571.822,...,0,0,0,0,7.1046,15.4863,15.4863,0,0.271806,NaN


In [ ]:
                  
                    
            #zFile = ZipFile(fileName, 'r')
            #pd.read_table(zFile.open('PlusFlow.dbf'))
                
        #if 'Hydrography' in fileName:
        #   subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + fileName)
        
        #Gave error BadZipFile: File is not a zip file
        #mysock = urllib.request.urlopen(fileUrl)
        #memfile = io.BytesIO(mysock.read())
        #ZipFile.infolist(memfile)
        #with ZipFile(memfile, 'r') as ftpl:
            #ftpl.namelist()
            #for file in flist:
            #    test = ftpl.extract(file)
            #    print (test)
      
        #Gave error BadZipFile: File is not a zip file
        #r = requests.get(fileUrl)
        #with closing(r), ZipFile(io.BytesIO(r.content)) as archive:
        #    print({member.filename: archive.read(member) for member in archive.infolist()})

In [15]:
print (finalurl)

https://gc2.datadistillery.org/api/v1/sql/bcb&q=insert into nhd.nhdplusv2_plusflowlinevaa                 (comid, fdate) VALUES                 ('939030274' ,'2012-01-06')


In [ ]:
for item in nhdFiles['items']:
    for tag in item['tags']:
        if 'Reg' in tag['name']:
            region = tag['name']
            print (region)

In [ ]:
print (nhdFiles)

In [ ]:
for item:
    for file in files:
        unzip file into memory
        if fileName = Flowline*
            convert correct file to dataframe
            
        
            

In [35]:
# Set up the actions/targets for this particular instance
thisRun = {}
thisRun["instance"] = "DataDistillery"
thisRun["db"] = "BCB"
thisRun["baseURL"] = gc2.sqlAPI(thisRun["instance"],thisRun["db"])
thisRun["schema"] = "sfr"
thisRun["commitToDB"] = False

#Query comids already in the table
q_recordToSearch = "SELECT comid as lookup FROM nhd.nhdplusv2_plusflowlinevaa " 
recordToSearch = requests.get(thisRun["baseURL"]+"&q="+q_recordToSearch).json()
p = recordToSearch['features']
lookup = []
#for record returned, add record to list
for f in p:
    string = f['properties']['lookup']
    if string.startswith('"') and string.endswith('"'):
        string = string[1:-1]  #For some reason these values are coming back with double quotes, this removes outer quote
        lookup.append(string)
lenList = len(lookup)
cntExisting = 0 
existingList = []
    

In [37]:
print (lookup)

[]
